In [ ]:
import pandas as pd
import XPER

from pathlib import Path
from src.utils.models_pkl import load_pickle
from src.modeling.create_data_split import split_data
from config.config_modeling import CAT_COLS

In [ ]:
OUT_PATH = Path("../data/data.csv")
df = pd.read_csv(OUT_PATH)

In [ ]:
MODEL_PATH = Path("../models/XGB.pkl")
model = load_pickle(MODEL_PATH)

In [ ]:
model

In [ ]:
data = split_data(cols=CAT_COLS, df=df)

In [ ]:
data["train"][0]

In [ ]:
X_train, y_train, X_val, y_val = data["train"][0], data["train"][1], data["val"][0], data["val"][1]

In [ ]:
from XPER.compute.Performance import ModelPerformance

# Define the evaluation metric(s) to be used
XPER = ModelPerformance(X_train, y_train, X_val, y_val, model)

# Evaluate the model performance using the specified metric(s)
PM = XPER.evaluate(["AUC"])

# Print the performance metrics
print("Performance Metrics: ", round(PM, 3))

In [ ]:
# Calculate XPER values for the model's performance
XPER_values = XPER.calculate_XPER_values(["Precision"])
# AUC takes much longer than Precision! For illustration purpose, it is better to choose precision.

In [ ]:
# Calculate XPER values for the model's performance
XPER_values = XPER.calculate_XPER_values(["AUC"], kernel=False)